## Problem: Write a GAN

### Problem Statement
Implement a **Generative Adversarial Network (GAN)** by completing the required sections. The GAN consists of a **Generator** that creates fake data and a **Discriminator** that classifies data as real or fake.

### Requirements
1. **Define the Generator Class**:
   - **Purpose**: Generate fake data that mimics the real data distribution.
   - **Layers**:
     - Start with a fully connected layer to map the latent space (random noise) to a higher-dimensional space.
     - Use activation functions like `ReLU` to introduce non-linearity.
     - Add additional layers to process the data and refine its structure.
     - The final layer should output data in the target shape. Use an activation function like `Tanh` for scaling.
   - **Forward Pass**: Implement the forward method to pass the input through the defined layers.

2. **Define the Discriminator Class**:
   - **Purpose**: Classify data as real or fake.
   - **Layers**:
     - Use fully connected layers to process the input and extract features.
     - Apply activation functions like `LeakyReLU` to prevent dead neurons and stabilize training.
     - The final layer should output a single probability (real or fake) using a `Sigmoid` activation.
   - **Forward Pass**: Implement the forward method to process the input through the layers.

3. **Train the GAN**:
   - Alternate between training the Generator and Discriminator.
   - Use binary cross-entropy loss for both models.
   - Monitor the loss and generated samples during training.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    
# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [3]:
# Generate synthetic data for training
torch.manual_seed(42)
real_data = torch.rand(100, 1) * 2 - 1  # 100 samples in the range [-1, 1]

# Initialize models, loss, and optimizers
latent_dim = 10
data_dim = 1
G = Generator(latent_dim, data_dim)
D = Discriminator(data_dim)

criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=0.001)
optimizer_D = optim.Adam(D.parameters(), lr=0.001)

In [4]:
# Training loop
epochs = 1000
for epoch in range(epochs):
    # Train Discriminator
    latent_samples = torch.randn(real_data.size(0), latent_dim)
    fake_data = G(latent_samples).detach()
    real_labels = torch.ones(real_data.size(0), 1)
    fake_labels = torch.zeros(real_data.size(0), 1)

    optimizer_D.zero_grad()
    real_loss = criterion(D(real_data), real_labels)
    fake_loss = criterion(D(fake_data), fake_labels)
    loss_D = real_loss + fake_loss
    loss_D.backward()
    optimizer_D.step()

    # Train Generator
    latent_samples = torch.randn(real_data.size(0), latent_dim)
    fake_data = G(latent_samples)
    optimizer_G.zero_grad()
    loss_G = criterion(D(fake_data), real_labels)
    loss_G.backward()
    optimizer_G.step()

    # Log progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

Epoch [100/1000] - Loss D: 0.7453, Loss G: 1.3754
Epoch [200/1000] - Loss D: 0.9934, Loss G: 0.9999
Epoch [300/1000] - Loss D: 1.5029, Loss G: 0.7750
Epoch [400/1000] - Loss D: 1.5478, Loss G: 0.6311
Epoch [500/1000] - Loss D: 1.2611, Loss G: 0.9484
Epoch [600/1000] - Loss D: 1.3900, Loss G: 0.6522
Epoch [700/1000] - Loss D: 1.3911, Loss G: 0.7018
Epoch [800/1000] - Loss D: 1.3909, Loss G: 0.6861
Epoch [900/1000] - Loss D: 1.3868, Loss G: 0.6897
Epoch [1000/1000] - Loss D: 1.3878, Loss G: 0.6974


In [5]:
# Generate new samples with the trained Generator
latent_samples = torch.randn(5, latent_dim)
with torch.no_grad():
    generated_data = G(latent_samples)
    print(f"Generated data: {generated_data.tolist()}")

Generated data: [[-0.9427759647369385], [0.45343008637428284], [0.3479097783565521], [0.43589913845062256], [-0.6375918388366699]]
